# 5-1: Conv Layers

### Code.5-1-1: Shapes of Conv Layers

In [2]:
import tensorflow as tf 
from tensorflow.keras.layers import Conv2D

N, n_H, n_W, n_C = 1, 28, 28, 3
n_filter = 10    # filter number
f_size = 3       # filter size 

images = tf.random.uniform(minval = 0, maxval=1, 
                           shape = ((N, n_H, n_W, n_C)))

conv = Conv2D(filters = n_filter, kernel_size = f_size)

y = conv(images)

W, B = conv.get_weights()

print(images.shape) # (1, 28, 28, 3)  image 갯수, image 높이, image 넓이, image channel
print(W.shape)      # (3, 3, 3, 10)   filter 높이, filter 넓이, filter channel, filter number
print(B.shape)      # (10,)           bias vector (filter number)

# n_H - f_size + 1, n_W - f_size + 1
print(y.shape)      # (1, 26, 26, 10) image 갯수, image 높이, image 넓이, filter number

(1, 28, 28, 3)
(3, 3, 3, 10)
(10,)
(1, 26, 26, 10)


### Code.5-1-2: Correlation Calculation

In [20]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D

N, n_H, n_W, n_C = 1, 5, 5, 3
n_filter = 1    # filter number
f_size = 3       # filter size 

images = tf.random.uniform(minval = 0, maxval=1, 
                           shape = ((N, n_H, n_W, n_C)))

conv = Conv2D(filters = n_filter, kernel_size = f_size)

y_tf = conv(images)                             # squeeze
print("y(tensorflow):\n", y_tf.numpy().squeeze()) # 이미지 갯수, 채널을 없애고 이미지 크기만을 출력
W, B = conv.get_weights()

images = images.numpy().squeeze()
W = W.squeeze()
# print(images.shape)
# print(W.shape)
# print(B.shape)

y_man = np.zeros(shape = (n_H - f_size + 1, n_W - f_size + 1))
for i in range(n_H - f_size + 1):
    for j in range(n_W - f_size + 1):
        window = images[i:i+f_size, j:j+f_size, :]
        y_man[i, j] = np.sum(window * W) + B

print("Y(manual):\n", y_man)


y(tensorflow):
 [[-0.21994977 -1.2494121  -1.2107842 ]
 [-1.3574984  -1.0456573  -0.7627469 ]
 [-0.36140314 -0.9816696  -0.9732243 ]]
Y(manual):
 [[-0.21994974 -1.24941206 -1.21078432]
 [-1.35749829 -1.0456574  -0.76274681]
 [-0.36140314 -0.9816696  -0.97322422]]


# Code.5-2: Conv Layer with Filters

### Code.5-2-1: Shapes with Filters

In [24]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D

N, n_H, n_W, n_C = 32, 28, 28, 3
n_filter = 5
k_size = 3      # kernel size

images = tf.random.uniform(minval = 0, maxval = 1,
                           shape = (N, n_H, n_W, n_C))

conv = Conv2D(filters = n_filter, kernel_size = k_size)

Y = conv(images)

W, B = conv.get_weights()

print("Input Image: {}".format(images.shape))
print("W/B: {} / {}".format(W.shape, B.shape))
print("Output Image: {}".format(Y.shape))

# Input Image: (32, 28, 28, 3)      image갯수, 이미지 높이, 이미지 넓이, 채널 수
# W/B: (3, 3, 3, 5) / (5,)          filter 높이, filter 넓이, filter channel, filter number
# Output Image: (32, 26, 26, 5)     image갯수, 이미지 높이, 이미지 넓이, filter number



Input Image: (32, 28, 28, 3)
W/B: (3, 3, 3, 5) / (5,)
Output Image: (32, 26, 26, 5)


### Code.5-2-2: Computations with Filters

In [59]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D

N, n_H, n_W, n_C = 1, 5, 5, 3
n_filter = 3
k_size = 4


images = tf.random.uniform(minval = 0, maxval = 1,
                           shape = (N, n_H, n_W, n_C))

# Forward Propagation(Tensorflow)
conv = Conv2D(filters = n_filter, kernel_size = k_size)
Y = conv(images)
# print(Y.numpy())
# print(Y.shape) #(1, 2, 2, 3) 3 -> R, G, B
#           R           G           B
# [[[[-0.60188156  0.9526543  -0.46114743]
#    [ 0.02425398  0.3703008   0.02022844]]

#   [[ 0.00590422  0.5366116  -0.18528454]
#    [-0.33464622  0.42195326 -0.5980505 ]]]]
# 보기 쉽게 변환
Y = np.transpose(Y.numpy().squeeze(), (2, 0, 1))
print("Y(Tensorflow): \n", Y)
#  [[[-0.60188156  0.02425398]      R
#   [ 0.00590422 -0.33464622]]

#  [[ 0.9526543   0.3703008 ]       G
#   [ 0.5366116   0.42195326]]

#  [[-0.46114743  0.02022844]       B
#   [-0.18528454 -0.5980505 ]]]

W, B = conv.get_weights()



# Forward Propagation(Manual)
images = images.numpy().squeeze()

Y_man = np.zeros(shape=(n_H - k_size + 1, n_W - k_size + 1, n_filter))
for c in range(n_filter):
    c_W = W[:, :, :, c] # c번째 필터 데이터 가져오기
    c_b = B[c]          # c번째 Bias 값 가져오기

    for h in range(n_H - k_size + 1):
        for w in range(n_W - k_size + 1):
            window = images[h:h + k_size, w:w + k_size, :]
            conv = np.sum(window * c_W) + c_b

            Y_man[h, w, c] = conv
print("\nY(Manual): \n", np.transpose(Y_man, (2, 0, 1)))


Y(Tensorflow): 
 [[[ 0.6156004   0.657022  ]
  [ 0.70373636  0.81284726]]

 [[ 0.6973136  -0.00518596]
  [ 0.29950017  0.83726317]]

 [[-1.2518162  -1.0487965 ]
  [-1.0345153  -0.52161324]]]

Y(Manual): 
 [[[ 0.61560035  0.657022  ]
  [ 0.70373631  0.8128469 ]]

 [[ 0.69731367 -0.00518593]
  [ 0.2995002   0.83726323]]

 [[-1.25181627 -1.04879653]
  [-1.03451526 -0.52161324]]]


In [42]:
import numpy as np

images = np.random.randint(low = 0, high = 10, size = (2, 3 ,4))
for c in range(4):                                   # 0  1  2  번째 차원     
    print(images[:, :, c])
                          # 2번째 차원을 0번쨰로 이동
images = np.transpose(images, (2, 0, 1))
                             # 0번째 차원을 1번쨰로 
                                  # 1번쨰 차원을 2번쨰로

print('\n\n',images.shape)

for c in range(4):
    print(images[c, :, :])

[[4 0 1]
 [5 4 3]]
[[2 1 8]
 [7 0 3]]
[[5 9 4]
 [4 5 7]]
[[7 2 9]
 [3 4 9]]


 (4, 2, 3)
[[4 0 1]
 [5 4 3]]
[[2 1 8]
 [7 0 3]]
[[5 9 4]
 [4 5 7]]
[[7 2 9]
 [3 4 9]]


# 5-3: Conv Layers with Activation Functions

In [62]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D

N, n_H, n_W, n_C = 1, 5, 5, 3
n_filter = 3
k_size = 4


images = tf.random.uniform(minval = 0, maxval = 1,
                           shape = (N, n_H, n_W, n_C))

# Forward Propagation(Tensorflow)
conv = Conv2D(filters = n_filter, kernel_size = k_size, activation = 'sigmoid')
Y = conv(images)
Y = np.transpose(Y.numpy().squeeze(), (2, 0, 1))
print("Y(Tensorflow): \n", Y)

W, B = conv.get_weights()



# Forward Propagation(Manual)
images = images.numpy().squeeze()

Y_man = np.zeros(shape=(n_H - k_size + 1, n_W - k_size + 1, n_filter))
for c in range(n_filter):
    c_W = W[:, :, :, c]
    c_b = B[c]

    for h in range(n_H - k_size + 1):
        for w in range(n_W - k_size + 1):
            window = images[h:h + k_size, w:w + k_size, :]
            conv = np.sum(window * c_W) + c_b
            conv = 1 / (1 + np.exp(-conv))

            Y_man[h, w, c] = conv
print("\nY(Manual): \n", np.transpose(Y_man, (2, 0, 1)))


Y(Tensorflow): 
 [[[0.38668457 0.5238226 ]
  [0.38520205 0.38785785]]

 [[0.46648622 0.3810928 ]
  [0.3796222  0.34227487]]

 [[0.36707437 0.5922849 ]
  [0.55133057 0.33311564]]]

Y(Manual): 
 [[[0.3866846  0.5238226 ]
  [0.38520195 0.38785786]]

 [[0.46648617 0.38109272]
  [0.37962219 0.34227485]]

 [[0.3670744  0.5922849 ]
  [0.5513305  0.33311567]]]


# 5-4: Models with Conv Layers

### Code.5-4-1: Models

In [68]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
            
n_neurons = [10, 20, 30] # filter 갯수

model = Sequential()
model.add(Conv2D(filters=n_neurons[0], kernel_size=3, activation = 'relu'))
model.add(Conv2D(filters=n_neurons[1], kernel_size=3, activation = 'relu'))
model.add(Conv2D(filters=n_neurons[2], kernel_size=3, activation = 'relu'))

x = tf.random.normal(shape = (32, 28, 28, 3))
predictions = model(x)

print("Input: {}".format(x.shape))
print("Output: {}".format(predictions.shape))

for layer in model.layers:
    W, B = layer.get_weights()
    print("Weights: {}, Biases: {}".format(W.shape, B.shape))
    
print("============================")
trainable_variables = model.trainable_variables
for train_var in trainable_variables:
    print("Shape: {}".format(train_var.shape))

Input: (32, 28, 28, 3)
Output: (32, 22, 22, 30)
Weights: (3, 3, 3, 10), Biases: (10,)
Weights: (3, 3, 10, 20), Biases: (20,)
Weights: (3, 3, 20, 30), Biases: (30,)
Shape: (3, 3, 3, 10)
Shape: (10,)
Shape: (3, 3, 10, 20)
Shape: (20,)
Shape: (3, 3, 20, 30)
Shape: (30,)


### Code.5-4-2: Models with Model sub-classing

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D

n_neurons = [10, 20, 30]

class TestModel(Model):
    def __init__(self):
        super(TestModel, self).__init__()
        global n_neurons

        self.conv_layers = []
        for n_neuron in n_neurons:
            self.conv_layers.append(Conv2D(filters = n_neuron, kernel_size = 3, activation = 'relu'))

    def __call__(self, x):
        print("Input ", x.shape)

        print("\n===== Conv Layers =====\n")

        for conv_layer in self.conv_layers:
            x = conv_layer(x)
            W, B = conv_layer.get_weights()
            print("W/B: {}/{}".format(W.shape, B.shape))
            print("X: {}\n".format(x.shape))
        return x
    
model = TestModel()
x = tf.random.normal(shape = (32, 28, 28, 3))
predictions = model(x)


print("Input: {}".format(x.shape))
print("Output: {}".format(predictions.shape))

for layer in model.layers:
    W, B = layer.get_weights()
    print("Weights: {}, Biases: {}".format(W.shape, B.shape))
    
print("============================")
trainable_variables = model.trainable_variables
for train_var in trainable_variables:
    print("Shape: {}".format(train_var.shape))




class TestModel(Model):
    def __init__(self):
        super(TestModel, self).__init__()
        global n_neurons

        self.conv1 = Conv2D(filters = n_neurons[0], kernel_size = 3, activation = 'relu')
        self.conv2 = Conv2D(filters = n_neurons[1], kernel_size = 3, activation = 'relu')
        self.conv3 = Conv2D(filters = n_neurons[2], kernel_size = 3, activation = 'relu')

    def __call__(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)

        return x

Metal device set to: Apple M1 Pro

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB

Input  (32, 28, 28, 3)

===== Conv Layers =====

W/B: (3, 3, 3, 10)/(10,)
X: (32, 26, 26, 10)

W/B: (3, 3, 10, 20)/(20,)
X: (32, 24, 24, 20)

W/B: (3, 3, 20, 30)/(30,)
X: (32, 22, 22, 30)

Input: (32, 28, 28, 3)
Output: (32, 22, 22, 30)
Weights: (3, 3, 3, 10), Biases: (10,)
Weights: (3, 3, 10, 20), Biases: (20,)
Weights: (3, 3, 20, 30), Biases: (30,)
Shape: (3, 3, 3, 10)
Shape: (10,)
Shape: (3, 3, 10, 20)
Shape: (20,)
Shape: (3, 3, 20, 30)
Shape: (30,)
